# 1. Data Preprocessing & Feature Engineering

In this notebook, we will:
1. Load the raw transaction data.
2. Downsample to 50K records for performance.
3. Engineer features: `Amount_log`, `Txn_per_hour` (Velocity).
4. Export the cleaned dataset.

In [ ]:
import pandas as pd
import numpy as np
import os

# Configuration
INPUT_FILE = '../data/raw_transactions.csv'
OUTPUT_FILE = '../data/cleaned_transactions.csv'
DOWNSAMPLE_SIZE = 50000

# Check if data exists, if not, warn user (or rely on synthetic generator which should have been run)
if not os.path.exists(INPUT_FILE):
    print(f"Data file {INPUT_FILE} not found. Please run generate_data.py or place the dataset in the data folder.")

In [ ]:
# 1. Load Data
try:
    df = pd.read_csv(INPUT_FILE)
    print(f"Data loaded. Shape: {df.shape}")
except FileNotFoundError:
    # Fallback for demonstration if user hasn't generated data yet
    print("Error: File not found.")
    df = pd.DataFrame()

In [ ]:
# 2. Downsample to 50K (Stratified to maintain Fraud ratio or simple random)
# User requested downsampling to 50K. We'll use simple random sampling but ensure we don't lose all frauds if they are rare.
# Actually, for fraud detection, usually we want to keep ALL frauds and downsample ONLY non-frauds.
# But for simplicity and strict adherence to "Downsample to 50K transactions", we will sample 50k random rows.

if len(df) > DOWNSAMPLE_SIZE:
    # Stratified sampling to preserve class distribution
    df = df.groupby('Class', group_keys=False).apply(lambda x: x.sample(int(np.rint(DOWNSAMPLE_SIZE*len(x)/len(df))))).sample(frac=1).reset_index(drop=True)
    print(f"Downsampled to {len(df)} rows.")
else:
    print("Dataset smaller than target, using all rows.")

In [ ]:
# 3. Feature Engineering

# Log Scale Amount
df['Amount_log'] = np.log1p(df['Amount'])

# Transaction Frequency per User (Velocity)
# Note: The original Kaggle dataset implies PCA features only, but we added a synthetic 'UserID' in our generation script
# or we assume the user has a dataset with UserID. If 'UserID' is missing, we create a dummy one for the sake of the logic.
if 'UserID' not in df.columns:
    print("UserID column missing. Generating dummy UserIDs for feature engineering demonstration.")
    df['UserID'] = np.random.randint(1, 5000, df.shape[0])

# Calculate Velocity: Transactions per hour (or just count per user as proxy for frequency)
# A more robust way: Count transactions by this user (overall or in a window)
df['Txn_per_user'] = df.groupby('UserID')['Time'].transform('count')

# Example High Risk Flag: High amount & High frequency
df['High_Risk_Flag'] = ((df['Amount_log'] > df['Amount_log'].quantile(0.95)) & 
                        (df['Txn_per_user'] > df['Txn_per_user'].quantile(0.95))).astype(int)

print("Feature Engineering Complete.")
print(df[['Amount', 'Amount_log', 'UserID', 'Txn_per_user', 'High_Risk_Flag']].head())

In [ ]:
# 4. Export
df.to_csv(OUTPUT_FILE, index=False)
print(f"Cleaned data saved to {OUTPUT_FILE}")